In [ ]:
import os
import gc
import time

from myModel import Mymodel

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as td

print(tf.__version__)

In [ ]:
csvdata = './dataset/dictonary.csv'
imgDir = './dataset/imges'

In [ ]:
dictonary = []
labels = []
imagePath = []
is_similar = []

df =pd.read_csv(csvdata)

for index,row in df.iterrows():
    labels.append(int(row['id']))
    dictonary.append(row['text'])
    is_similar.append(row['is_similar'])

dictonary, labels ,is_similar = shuffle(dictonary,
                                          labels,
                                          is_similar,
                                          random_state=1)
for name in labels:
    imgPath = os.path.join(imgDir,str(name)+'.jpeg')
    if os.path.exists(imgPath):
        imagePath.append(imgPath) 
print(len(is_similar),len(dictonary),len(imagePath))  

In [ ]:
model = Mymodel()
model.prepare_tokenizer(dictonary) 
mod = model.model() 
mod.summary() 

In [ ]:
imagePath_train, imagePath_val, dictonary_train, dictonary_val,is_similar_train, is_similar_val = \
train_test_split(imagePath,dictonary,is_similar,test_size=0.2,random_state=0)
img_features_train, text_vector_train =model.dataPost_processing(imagePath_train,dictonary_train)
gc.collect()
img_features_val, text_vector_val =model.dataPost_processing(imagePath_val,dictonary_val)
gc.collect()
is_similar_train = np.array(is_similar_train) 
is_similar_val = np.array(is_similar_val) 

In [ ]:
print(img_features_train.shape,text_vector_train.shape,is_similar_train.shape)
print(img_features_val.shape,text_vector_val.shape,is_similar_val.shape)

In [ ]:
mod.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
STAMP = 'lstm_%d_%d_%.2f_%.2f' % (model.number_lstm_units, model.number_dense_units, model.rate_drop_lstm, model.rate_drop_dense)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
checkpoint_dir = model.model_save_directory + 'checkpoints/' + str(int(time.time())) + '/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)  
bst_model_path = checkpoint_dir + STAMP + '.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=checkpoint_dir + "logs/{}".format(time.time()))

mod.fit([text_vector_train,img_features_train], is_similar_train,
          validation_data=([text_vector_val,img_features_val], is_similar_val),
          epochs=20, batch_size=16, shuffle=True,
          callbacks=[early_stopping, model_checkpoint, tensorboard])